# Test Gauss-Newton plane-based registration

In [ ]:
%matplotlib widget
import numpy as np
from plotly.subplots import make_subplots
import os

import planeslam.io as io
from planeslam.general import plot_3D_setup, color_legend
from planeslam.scan import pc_to_scan
from planeslam.registration import extract_corresponding_features, get_correspondences, residual, jacobian, so3_expmap, se3_expmap
from planeslam.geometry.util import skew

os.environ['KMP_DUPLICATE_LIB_OK']='True'

%load_ext autoreload
%autoreload 2

Read in airsim LiDAR and pose data

In [ ]:
# Read in point cloud data
binpath = os.path.join(os.getcwd(), '..', '..', 'data', 'airsim', 'blocks_60_samples_loop_closure', 'lidar', 'Drone0')
PC_data = io.read_lidar_bin(binpath)

In [ ]:
# Read in ground-truth poses (in drone local frame)
posepath = os.path.join(os.getcwd(), '..', '..', 'data', 'airsim', 'blocks_60_samples_loop_closure', 'poses', 'Drone0')
drone_positions, drone_orientations = io.read_poses(posepath)

Extract planes 

In [ ]:
num_scans = len(PC_data)
scans = num_scans * [None]
scans_transformed = num_scans * [None]
for i in range(num_scans):
    scans[i] = pc_to_scan(PC_data[i])

Get correspondences

In [ ]:
# Plot 2 scans
idx_1 = 1
idx_2 = 0
source = scans[idx_1]
target = scans[idx_2]

fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'surface'}, {'type': 'surface'}]])

for t in source.plot_trace(show_normals=True):
    fig.add_trace(t, row=1, col=1)

for t in target.plot_trace(show_normals=True):
    fig.add_trace(t, row=1, col=2)

fig.update_layout(width=1600, height=700, scene=dict(aspectmode='data'), scene2=dict(aspectmode='data'))
fig.show()

correspondences = get_correspondences(source, target)
print(correspondences)

In [ ]:
n_s, d_s, n_t, d_t = extract_corresponding_features(source, target, correspondences)

In [ ]:
correspondences

SO(3) Optimization

In [ ]:
from planeslam.registration import so3_jacobian
from planeslam.registration import so3_residual

# Rotation estimation
R_hat = np.eye(3)

n_iters = 5
lmbda = 1e-8
mu = 1.0

for i in range(n_iters):
    r, n_q = so3_residual(R_hat, n_s, n_t)
    #print("loss: ", np.linalg.norm(r)**2)
    J = so3_jacobian(n_q)
    dw = - mu * np.linalg.inv(J.T @ J + lmbda*np.eye(3)) @ J.T @ r
    R_hat = so3_expmap(dw.flatten()) @ R_hat

r, _ = so3_residual(R_hat, n_s, n_t)
print("final rotation loss: ", np.linalg.norm(r)**2)

# Translation estimation
Rn_s = (R_hat @ n_s.reshape((3, -1), order='F'))
t_hat = np.linalg.lstsq(Rn_s.T, d_s - d_t, rcond=None)[0]

In [ ]:
R_hat

In [ ]:
t_hat

In [ ]:
from planeslam.geometry.util import quat_to_rot_mat

R_1 = quat_to_rot_mat(drone_orientations[idx_1])
R_2 = quat_to_rot_mat(drone_orientations[idx_2])
R_gt = R_2 @ R_1.T

t_gt = drone_positions[idx_2] - drone_positions[idx_1]

In [ ]:
R_gt

In [ ]:
t_gt

Test SE(3) Gauss-Newton registration

In [ ]:
# Initial transformation
# t = np.array([0, 1, 0])[:,None]
# u = np.array([1, 0, 0])[:,None]
# theta = 0.1
# #q = np.vstack((t, theta*u))
# q = np.random.randn(6,1)
T = np.eye(4)

# Gauss-Newton
n_iters = 20
lmbda = 0.0
mu = 1.0

for i in range(n_iters):
    r, n_q = residual(n_s, d_s, n_t, d_t, T)
    print("loss: ", np.linalg.norm(r)**2)
    J = jacobian(n_s, n_q)
    dv = -mu * np.linalg.inv(J.T @ J + lmbda*np.eye(6)) @ J.T @ r
    T = se3_expmap(dv.flatten()) @ T

# # Gauss-Newton
# n_iters = 10
# lmbda = 1e-8
# mu = 5e-1

# for i in range(n_iters):
#     r, n_q = residual(n_s, d_s, n_t, d_t, q)
#     print("loss: ", np.linalg.norm(r)**2)
#     J = jacobian(n_s, n_q)
#     dq = - mu * np.linalg.inv(J.T @ J + lmbda * np.eye(6)) @ J.T @ r

Test with two planes

In [ ]:
from planeslam.geometry.plane import BoundedPlane
from planeslam.registration import so3_expmap, se3_expmap
from planeslam.scan import Scan
import copy

V1 = np.array([[-1, 1, -1],
              [1, 1, -1],
              [1, 1, 1],
              [-1, 1, 1]])
V2 = np.array([[-1, -1, -1],
              [-1, 1, -1],
              [-1, 1, 1],
              [-1, -1, 1]])
P = Scan([BoundedPlane(V1), BoundedPlane(V2)])

# Ground-truth transformation
t = np.array([0, -0.5, 0])[:,None]
u = np.array([0, 1, 0])[:,None]
theta = np.pi/12
q = np.vstack((t, theta*u))

R = so3_expmap(q[3:].flatten())

print("t: ", t)
print("R: ", R)

In [ ]:
# P is source, Q is target
Q = copy.deepcopy(P)
Q.transform(R, t.flatten())

In [ ]:
ax = plot_3D_setup()
P.plot(ax, color='b')
Q.plot(ax, color='r')

In [ ]:
# n_s = P.normal 
# d_s = np.dot(P.normal.flatten(), P.center)
# n_t = Q.normal 
# d_t = np.dot(Q.normal.flatten(), Q.center)
correspondences = [(0,0), (1,1)]
n_s, d_s, n_t, d_t = extract_corresponding_features(P, Q, correspondences)

In [ ]:
R = np.eye(3)

# Gauss-Newton
n_iters = 5
lmbda = 1e-8
mu = 1.0

for i in range(n_iters):
    r, n_q = so3_residual(R, n_s, n_t)
    print("loss: ", np.linalg.norm(r)**2)
    J = so3_jacobian(n_q)
    dw = - mu * np.linalg.inv(J.T @ J + lmbda*np.eye(3)) @ J.T @ r
    R = so3_expmap(dw.flatten()) @ R

In [ ]:
R

In [ ]:
# Initial transformation
# t = np.array([0, 1, 0])[:,None]
# u = np.array([1, 0, 0])[:,None]
# theta = 0.1
# q = np.vstack((t, theta*u))
T = np.eye(4)

# Gauss-Newton
n_iters = 20
lmbda = 1e-8
mu = 0.5

for i in range(n_iters):
    r, n_q = residual(n_s, d_s, n_t, d_t, T)
    print("loss: ", np.linalg.norm(r)**2)
    J = jacobian(n_s, n_q)
    dv = - mu * np.linalg.inv(J.T @ J + lmbda*np.eye(6)) @ J.T @ r
    T = se3_expmap(dv.flatten()) @ T

In [ ]:
# Extract R and t from q
# t_est = q[:3]
# R_est = expmap(q[3:].flatten())
# print("t_est: ", t_est)
# print("R_est: ", R_est)

# Apply transformation to source
P_T = copy.copy(P)
P_T.transform(R, t.flatten())

In [ ]:
plot_P = np.vstack((np.eye(3), -np.eye(3)))
ax = plot_3D_setup(P=plot_P)
P_T.plot(ax, color='b')
Q.plot(ax, color='r')

Rotation only

In [ ]:
from planeslam.geometry.util import skew
from planeslam.registration import so3_expmap

In [ ]:
def residual(R, n_s, n_t):
    n_q = (R @ n_s.reshape((3, -1), order='F')).reshape((-1, 1), order='F')
    return n_q - n_t, n_q

def jacobian(n_q):
    N = int(len(n_q) / 3)

    J = np.empty((3*N,3))
    for i in range(N):
        Rn_i = n_q[3*i:3*i+3].flatten()
        J[3*i:3*i+3,:] = -skew(Rn_i)
    
    return J

In [ ]:
n_s = np.array([1,2,3, 1,2,1, -1,1,2])[:,None]

R_gt = so3_expmap([0, 0.5, 0])
n_t = (R_gt @ n_s.reshape((3, -1), order='F')).reshape((-1, 1), order='F')

In [ ]:
R = np.eye(3)
mu = 1.0
lmbda = 1e-5

for i in range(10):
    r, n_q = residual(R, n_s, n_t)
    print("loss: ", np.linalg.norm(r)**2)
    J = jacobian(n_q)
    dw = -mu * np.linalg.inv(J.T @ J + lmbda*np.eye(3)) @ J.T @ r
    R = so3_expmap(dw.flatten()) @ R

In [ ]:
R

In [ ]:
R_gt